In [1]:
from scipy.stats import (
    norm, binom, expon, t, pareto, ttest_1samp, ttest_ind, sem, uniform, bernoulli
)
from statsmodels.stats.api import CompareMeans, DescrStatsW
from statsmodels.stats.proportion import proportion_confint
import numpy as numpy
from seaborn import distplot
from matplotlib import pyplot
import seaborn
from tqdm.notebook import tqdm

import sys
sys.path.append('.')

import warnings
warnings.filterwarnings("ignore")
numpy.random.seed(42)

In [2]:
def inverse_plot_colorscheme():
    import cycler
    def invert(color_to_convert): 
        table = str.maketrans('0123456789abcdef', 'fedcba9876543210')
        return '#' + color_to_convert[1:].lower().translate(table).upper()
    update_dict = {}
    for key, value in pyplot.rcParams.items():
        if value == 'black':
            update_dict[key] = 'white'
        elif value == 'white':
            update_dict[key] = 'black'
    
    old_cycle = pyplot.rcParams['axes.prop_cycle']
    new_cycle = []
    for value in old_cycle:
        new_cycle.append({
            'color': invert(value['color'])
        })
    pyplot.rcParams.update(update_dict)
    pyplot.rcParams['axes.prop_cycle'] = cycler.Cycler(new_cycle)
    lec = pyplot.rcParams['legend.edgecolor']
    lec = str(1 - float(lec))
    pyplot.rcParams['legend.edgecolor'] = lec

In [3]:
inverse_plot_colorscheme()

# Ratio-метрики


В сегодняшней лекции мы узнаем, как работать с ratio-метриками.


> 📈 **Задача**
>
> Вы аналитик в Авито. Недавно к вам пришел продукт-менеджер, который захотел поменять цвет кнопки "написать сообщение" в объявлениях из синего в красный. По всем стандартным шаблонам.
>
> Вопрос: увеличит ли это конверсию в нажатие кнопки или конверсию в начатые чаты по объявлениям?

Давайте для примера посчитаем эту метрику для 4 пользователей:

| Id пользователя | Сколько раз зашел на страницу объявления? | Сколько раз начал общение в чате? |
|------|----------------------------------------|-----------------------|
| 1    | 5                                      | 5                     |
| 2    | 4                                      | 1                     |
| 3    | 0                                      | 0                     |
| 4    | 6                                      | 4                     |


Тогда у нас было 15 заходов на страницу объявлений и 10 объявлений, на которых произошло общение в чате. Интересующая нас конверсия в задаче равна 10/15 или 2/3 или 66%.


Хорошо, тогда для решения надо провести AB-тест: Половину пользователей в тест, другую в контроль. Например: пользователь 1 и 3 в контроль, а 2 и 4 в тест.

---

## T-test. Статистические единицы = заходы на страницу объявления.


Какой критерий мы знаем для сравнения конверсий в тесте и в контроле? T-test и хи-квадрат. Но надо понять, к каким выборкам их применять.


Пусть нашими статистическими единицами будут "заходы на страницу объявления": Если после этого началось общение в чате, то это 1, а если нет – то это 0. Тогда у нас получается следующая ситуация в тесте и в контроле:

<img src="https://github.com/dimalunin2016/pictures/blob/main/in_ration.png?raw=true" width="1000" height="300" />

Тогда: $M_O^C,\ M_O^T$ – случайные величины в тесте и в контроле, равные 1 или 0. $O$ внизу означает, что статистическая единица измерения - это число открытых объявлений юзером (opened), а $M$ – началась ли переписка в мессенджере (messenger).

Поставим гипотезу на языке математики: $H_0: \mathbb{E}\overline{M_O^C} = \mathbb{E} \overline{M_O^T}\ VS.\ H_1: \mathbb{E}\overline{M_O^C} < \mathbb{E} \overline{M_O^T}$. Ее должно быть легко проверить через T-test. Но не все так просто.

------

Можно ли тут использовать T-test для замера разницы конверсий? Давайте проверим. Что если вся разница между группами, задетектированная T-test, будет обусловлена разными пользователями, а не эффектом между тестом и контролем? Например, первый пользователь не любит звонить и всегда пишет в чат, а остальные нет. Тогда конверсия в контроле всегда будет больше.

Проверим на Монте-Карло:

- Для этого мы создадим функцию для генерации искусственного пользователя $i$.
    - На первом этапе мы сгенерируем, на какое число объявлений пользователь зашел на сайте. Пусть это число $O_i$, его мы сгенерируем из нормального распределения для простоты.
    - Вторым этапом сгенерируем конверсию $p_i$ нажатия на кнопку "написать сообщение".
    - Так для каждого пользователя получится выборка размера $O_i$ с конверсией $p_i$. Для ускорения работы будем считать, что число чатов будет ровно $[O_i \cdot p_i]$ (Целая часть от $O_i \cdot p_i$).
- При этом пользователи будут взяты из одного распределения: не будет никакой разницы между пользователями в тесте и в контроле.

In [15]:
def generate_incorrect_group_sample(group_size, p=0.3):
        
    # Сгенерируем число заходов на страницу объявлений для всех пользователей группы сразу.
    O_array = norm(loc=10, scale=1).rvs(size=group_size).astype(int)
    # Сгенерируем конверсию нажатия на кнопку "написать сообщение".
    p_array = uniform().rvs(size=group_size)
    
    # размер всей выборки равен Sum (O_i)
    sample_size = numpy.sum(O_array)
    # Число чатов.
    M_sum = numpy.sum((O_array * p_array).astype(int))
    
    # Неважно, в каком порядке будут стоять элементы
    return [1] * M_sum + [0] * (sample_size - M_sum)

In [16]:
numpy.random.seed(42)
mc_size = 1000
bad_cnt = 0
alpha = 0.05
group_size = 1000

for _ in tqdm(range(mc_size)):
    test_sample = generate_incorrect_group_sample(group_size)
    control_sample = generate_incorrect_group_sample(group_size)
    
    # Запускаю критерий и считаю p-value
    pvalue = ttest_ind(test_sample, control_sample, equal_var=True, alternative='two-sided').pvalue

    # Проверяю, что pvalue <= alpha
    bad_cnt += (pvalue <= alpha)

print(f"FPR: {round(bad_cnt / mc_size, 4)}")
print(f"CI={proportion_confint(count = bad_cnt, nobs = mc_size, alpha=0.05, method='wilson')}")

  0%|          | 0/1000 [00:00<?, ?it/s]

FPR: 0.263
CI=(0.23665685979107837, 0.2911570237310747)


FPR равен 62% вместо 5%. **В этом случае нельзя использовать t-test!**

Почему такое произошло? Потому что элементы в контроле **зависимы между собой (их совершил 1 пользователь)**! Это верно и для
теста. В этом случае, из-за ограничений критерия (он работает с независимыми элементами), нельзя использовать t-test! **Поэтому всегда помните о зависимостях в данных!** Казалось бы, простая задача про раскраску кнопки в другой цвет, но и она имеет подводные камни.

Чтобы лучше понять этот феномен давайте представим, что у нас 
- по одному пользователю в тесте и в контроле, где каждый из них зашел 10000 раз на объявление. 
- У первого конверсия в написание в чат равна 1, а у второго 0, причем всегда. Один боится звонков, а второй обожает общаться по телефону, поэтому в чат никогда не напишет.
- Тогда у нас в тесте конверсия будет 1, а в контроле 0. 
- T-test все отвергнет: для него будет 2 выборки размера 10000: одна из единиц, другая из нулей.
- А если бы мы поменяли местами пользователей, то было бы наоборот: контроль стал бы лучше теста.

Проблема состоит в том, что у нас всего 2 пользователя и t-test неприменим. А мы "размножили" их и получили выборки размера 10000 и применяем t-test. Мы просто неправильно работаем с этим критерием.

Но давайте посмотрим на условие задачи внимательнее.


----
## Решение 0 (бейзлайн). T-test. Статистические единицы = пользователи Авито.

Что на самом деле интересует нас в этой задаче? **Не конверсия из открытия объявления в чат, а прирост числа чатов!**

Поэтому давайте поменяем нашу гипотезу: вместо  $H_1: \mathbb{E}\overline{M_O^C} < \mathbb{E} \overline{M_O^T}$ (это изначальная постановка задачи) будем смотреть  $H_1: \mathbb{E}\overline{M^C} < \mathbb{E} \overline{M^T}$, где $M^C,\ M^T$ &mdash; среднее число заходов в мессенджер от пользователей в тесте и контроле. В этом случае мы смотрим среднее на юзера, а не на открытое объявление, как в предыдущем случае. А также избавляемся от зависимости в выборках: открытие чата одним пользователем никак не влияет на открытие чата другим пользователем.


Проверим, что замена валидна на Монте-Карло.

In [6]:
def generate_group_sample(group_size):
        
    # Сгенерируем число заходов на страницу объявлений для всех пользователей группы сразу.
    O_array = norm(loc=50, scale=5).rvs(size=group_size).astype(int)
    # Сгенерируем конверсию нажатия на кнопку "написать сообщение".
    p_array = uniform().rvs(size=group_size)
    
    # Число чатов у каждого пользователя [O * p].
    M_array = (O_array * p_array).astype(int)
        
    # Неважно, в каком порядке будут стоять элементы
    return {
        "O_array": O_array,
        "message_array": M_array
    }

In [7]:
numpy.random.seed(42)
mc_size = 1000
bad_cnt = 0
alpha = 0.05
group_size = 1000

for _ in tqdm(range(mc_size)):
    test_sample = generate_group_sample(group_size)
    control_sample = generate_group_sample(group_size)
    
    # Запускаю критерий и считаю p-value
    pvalue = ttest_ind(test_sample['message_array'], control_sample['message_array'],
                       equal_var=False, alternative='two-sided').pvalue
    # Проверяю, что pvalue <= alpha
    bad_cnt += (pvalue <= alpha)

print(f"FPR: {round(bad_cnt / mc_size, 4)}")
print(f"CI={proportion_confint(count = bad_cnt, nobs = mc_size, alpha=0.05, method='wilson')}")

  0%|          | 0/1000 [00:00<?, ?it/s]

FPR: 0.05
CI=(0.03813026239274881, 0.06531382024425081)


Да, в такой формулировке все верно.

------

Но нам этого мало. Ответьте на вопрос: какое максимальное число чатов может быть у любого юзера Авито? Непонятно. А если вы знаете, что он заходил на страницу лишь 1 объявления? Тогда вы очень легко можете ответить на вопрос: 1 чат.
А значит, число чатов может быть сильно связано с числом заходов на объявления у юзера. И нам не хочется терять эту информацию: если мы сможем ее использовать, то мы сможем создать более мощный критерий!

Поэтому давайте попытаемся решить задачу для изначальной постановки и сравним мощности текущего и нового решения.

----

## Решение 1: bootstrap

Давайте формально выпишем, что значит $H_1: \mathbb{E}\overline{M_O^C} < \mathbb{E} \overline{M_O^T}$. 
Пусть

- $O^C,\ O^T$ &mdash; число открытий объявлений пользователями в контроле и тесте.
- $M^C,\ M^T$ &mdash; число заходов в мессенджер пользователями в тесте и контроле.

$$\begin{align}
&H_0: \mathbb{E} \dfrac{\sum M^T}{\sum O^T} - \mathbb{E} \dfrac{\sum M^C}{\sum O^C} = 0\ vs.\ \\ &H_1: \mathbb{E} \dfrac{\sum M^T}{\sum O^T} - \mathbb{E} \dfrac{\sum M^C}{\sum O^C} > 0
\end{align}
$$

Мы подробно расписали, что такое числитель и знаменатель, чтобы лучше распознавать наши случайные величины в статистике.

В дальнейшем метрики типа $\dfrac{\sum X}{\sum Y}$ мы будем называть __ratio-метриками__. Кстати, обычный поюзерный T-test также можно расписать как Ratio: вместо $H_1: \mathbb{E}\overline{M^C} < \mathbb{E} \overline{M^T}$ можно написать $H_1: \mathbb{E} \dfrac{\sum M^T}{\sum U^T} - \mathbb{E} \dfrac{\sum M^C}{\sum U^C} > 0$, где $U^T, U^C$ – число юзеров в тесте и в контроле.

----

В итоге, нас интересует посчитать статистику $Z = \dfrac{\sum M_T}{\sum O_T} - \dfrac{\sum M_C}{\sum O_C}$ и построить доверительный интервал для нее. Давайте воспользуемся бутстрапом: он поможет в любой непонятной ситуации.



**Важно:** в данном случае надо помнить, что в бутстрапе надо семплировать одновременно, как число чатов, так и число заходов на объявление, потому что они зависимы от одного пользователя!

In [8]:
def generate_group_sample(group_size):
        
    # Сгенерируем число заходов на страницу объявлений для всех пользователей группы сразу.
    O_array = norm(loc=50, scale=5).rvs(size=group_size).astype(int)
    # Сгенерируем конверсию нажатия на кнопку "написать сообщение".
    p_array = uniform().rvs(size=group_size)
    
    # Число чатов у каждого пользователя [O * p].
    M_array = (O_array * p_array).astype(int)
        
    # Неважно, в каком порядке будут стоять элементы
    return {
        "O_array": O_array,
        "message_array": M_array
    }

In [9]:
def ratio_bootstrap(message_array, message_control, n_test, n_control, alpha=0.05):
    theta_func = lambda MT, MC, OT, OC: numpy.sum(MT, axis=1) / numpy.sum(OT, axis=1) - numpy.sum(MC, axis=1) / numpy.sum(OC, axis=1)
    
    
    B = 1000 # Чтобы ускорить работу
    batch_size = B // 20

    
    theta_asterisk_array = []

    test_size = len(message_array)
    control_size = len(message_control)

    test_inds_array = numpy.arange(0, test_size)
    control_inds_array = numpy.arange(0, control_size)
    for _ in range(0, B, batch_size):
        
        boot_t_inds = numpy.random.choice(test_inds_array, replace=True, size=(batch_size, test_size))
        boot_c_inds = numpy.random.choice(control_inds_array, replace=True, size=(batch_size, control_size))
        
        # Неверный способ
        # boot_MT = numpy.random.choice(message_array, replace=True, size=(batch_size, test_size))
        # boot_MC = numpy.random.choice(message_control, replace=True, size=(batch_size, control_size))
        # boot_OT = numpy.random.choice(n_test, replace=True, size=(batch_size, test_size))
        # boot_OC = numpy.random.choice(n_control, replace=True, size=(batch_size, control_size))
        boot_MT = message_array[boot_t_inds]
        boot_MC = message_control[boot_c_inds]
        boot_OT = n_test[boot_t_inds] # Берутся те же юзеры, которые были при семплировании boot_OT
        boot_OC = n_control[boot_c_inds] # Берутся те же юзеры, которые были при семплировании boot_OC
        
        theta_asterisk = theta_func(
            boot_MT, boot_MC, boot_OT, boot_OC
        ).ravel()
        assert len(theta_asterisk) == batch_size
        theta_asterisk_array = numpy.concatenate([theta_asterisk_array, theta_asterisk])
    left_theta_asterisk, right_theta_asterisk = numpy.quantile(theta_asterisk_array, [alpha / 2, 1 - alpha / 2])
    
    
    percentile_left_bound, percentile_right_bound = left_theta_asterisk, right_theta_asterisk
    return percentile_left_bound, percentile_right_bound

In [10]:
numpy.random.seed(8)

bad_cnt_correct_bootstrap = 0

for _ in tqdm(range(mc_size)):
    test_sample = generate_group_sample(group_size)
    control_sample = generate_group_sample(group_size)

  
    left_bound_correct, right_bound_correct = ratio_bootstrap(
        test_sample['message_array'], 
        control_sample['message_array'], 
        test_sample['O_array'], 
        control_sample['O_array']
    )
   
    bad_cnt_correct_bootstrap += ((left_bound_correct > 0) | (right_bound_correct < 0))

corr_fpr = round(bad_cnt_correct_bootstrap / mc_size, 4)
corr_left, corr_right = proportion_confint(count = bad_cnt_correct_bootstrap, 
                                           nobs = mc_size, alpha=0.05, method='wilson')
print(f"FPR, correct bootstrap: {round(corr_fpr, 4)},"\
      f" [{round(corr_left, 4)}, {round(corr_right, 4)}]")


  0%|          | 0/1000 [00:00<?, ?it/s]

FPR, correct bootstrap: 0.054, [0.0416, 0.0698]


Отлично, теперь мы построили корректный критерий для ratio-метрик!

----

## Зачем вообще нужны Ratio-метрики?

Давайте же теперь сравним мощности 2 критериев: T-test и бутстрапа для Ratio-метрики. Правда ли, что мощность стала больше, как мы и думали в начале?

Для того, чтобы сгенерировать АБ-тест чуть поменяем функцию генерации пользовательских выборок: добавим возможность изменять конверсии в тесте и в контроле.

In [11]:
def generate_group_sample_equal_cr(group_size, p=0.1):
        
    # Сгенерируем число заходов на страницу покупки для всех пользователей группы сразу.
    O_array = norm(loc=50, scale=5).rvs(size=group_size).astype(int)
    
    # Число покупок у каждого пользователя [N * p].
    p_array = norm(loc=p, scale=p / 10).rvs(group_size)
    p_array = numpy.maximum(p_array, 0)
    
    M_array = (O_array * p_array).astype(int)
        
    # Неважно, в каком порядке будут стоять элементы
    return {
        "O_array": O_array,
        "message_array": M_array
    }

In [12]:
numpy.random.seed(8)

power_ratio_cnt = 0
power_means_cnt = 0

for _ in tqdm(range(mc_size)):
    # Для лучшего эффекта возьмем, что числитель и знаменатель очень сильно зависят друг от друга;
    test_sample = generate_group_sample_equal_cr(group_size, p=0.101)
    control_sample = generate_group_sample_equal_cr(group_size, p=0.1)

    # Запускаю критерий и строю доверительный интервал
    left_bound_ratio, right_bound_ratio = ratio_bootstrap(
        test_sample['message_array'], 
        control_sample['message_array'], 
        test_sample['O_array'], 
        control_sample['O_array']
    )
    
    # Запускаю критерий и считаю p-value
    pvalue_means = ttest_ind(test_sample['message_array'], control_sample['message_array'],
                       equal_var=False, alternative='two-sided').pvalue
    
   
    power_ratio_cnt += (left_bound_ratio > 0)
    power_means_cnt += (pvalue_means < 0.05)


power_ratio = round(power_ratio_cnt / mc_size, 4)
ratio_left, ratio_right = proportion_confint(
    count = power_ratio_cnt, nobs = mc_size, alpha=0.05, method='wilson')
print(f"power, ratio: {round(power_ratio, 4)},"\
      f" [{round(ratio_left, 4)}, {round(ratio_right, 4)}]")


power_means = round(power_means_cnt / mc_size, 4)
means_left, means_right = proportion_confint(count = power_means_cnt, 
                                           nobs = mc_size, alpha=0.05, method='wilson')
print(f"power, means: {round(power_means, 4)},"\
      f" [{round(means_left, 4)}, {round(means_right, 4)}]")


  0%|          | 0/1000 [00:00<?, ?it/s]

power, ratio: 0.494, [0.4631, 0.525]
power, means: 0.328, [0.2996, 0.3577]


Мы видим, что мощность практически в 1.5 раза больше (а в реальности может быть и больше)! Отлично, гипотеза подтвердилась! Почему такое произошло?

- в ratio метрике в текущем примере вы сравниваете $p^T\ vs.\ p^C$ (так как $\dfrac{O \cdot p}{O} = p$) 
- А в t-test происходит сравнение $M^T\ vs.\ M^C$, или ${O^T \cdot p^T}\ vs.\ {O^C \cdot p^C}$, что является более шумной величиной из-за случайной величины O.

В реальных задачах все то же самое: вы убираете из данных часть шума, обусловленного знаменателем. Тем самым вы получаете менее шумную метрику.

----

**Менее важный аргумент**

- Вы можете лучше понять, как вы влияете на разные этапы "воронки". Например, вы так можете посмотреть, как вы вашим изменением повлияли на конверсию из захода на сайт в заход на страницу объявления, и еще посмотреть, как изменилась конверсия из страницы объявления в заход в чат. Тем самым вы можете лучше выделять проблемы для бизнеса в вашем изменении.



### Опасность в использовании ratio-метрики

Главная проблема ratio-метрики &mdash; вы добавляете знаменатель к вашей метрике, который также может поменяться от вашего изменения на сайте. И в этот момент результат $H_1: \mathbb{E} \dfrac{\sum M^T}{\sum O^T} - \mathbb{E} \dfrac{\sum M^C}{\sum O^C} > 0$ будет не согласован с $H_1: \mathbb{E} M^T -  \mathbb{E} M^C > 0$.

Например, в нашей задаче мы могли бы вместо покраса кнопки в красный цвет сделать неадекватное изменение: зашел в чат из объявления -> тебя заблокировали. Конверсия в тестовой группе была бы равна 1. Но вот число пользователей, а соответственно, и число заходов на страницу объявления сильно уменьшилось бы (все бы перестали пользоваться Авито).

Поэтому бывает полезно не просто замерить изменение ratio-метрики, но и дополнительно проверить знаменатель с помощью t-test! Так вы будете уверены, что вы стат. значимо не изменили знаменатель и данным можно верить. 

Поэтому, ваш план работы с ratio-метрикой таков:
- Если вы уверены, что не повлияли на знаменатель, то просто проверяете изменение ratio-метрики.
- Если же есть шанс затронуть знаменатель, то вы проверяете и ratio-метрику, и знаменатель. 

-------

## Решение 2: Линеаризация

Мы уже поняли, как можно решать задачу через бутстрап. Но его главный недостаток в том, что это очень медленный критерий, который также требует много памяти и ресурсов.

Представим, что вы устроились в новую компанию, где все плохо с АB-тестами: каждый делает их, как хочет, совершая при этом множество ошибок. Вы решаете создать единую систему для AB-тестов со следующими свойствами:
- Она унифицирует все AB-тесты в компании. Таким образом, сокращается число неверно созданных AB-тестов, если бы все аналитики сами проводили AB, сами собирали данные и применяли критерии, то любая ошибка в коде приводила бы к плачевным результатам.
- Сильно сокращает время на создание и анализ AB-теста, что также очень важно для крупных компаний.
- Подсвечивает, какие еще эксперименты запущены в компании, что помогает не проводить 2 противоречащих друг другу теста.

Это достаточно популярная задача для всех компаний. Например, в Авито также была реализована такая же система, названная "AB-central". Подробнее про нее можно прочесть на [хабре](https://habr.com/ru/company/avito/blog/454164/).

Так вот, в такой системе очень важно использовать быстрые критерии, требующие небольшого числа памяти. Поэтому для такой системы бутстрап не подходит. Что же тогда использовать?


**Часть 1. Многочлен Тейлора**

Давайте вспомним, что такое многочлен Тейлора и как с его помощью можно расписать любую функцию:

$f(x) = \sum_{k=0}^{n}\dfrac{f^{(k)}(x_0)}{k!}(x - x_0)^k + o((x - x_0)^{n}), x \rightarrow x_0$.

Если разложить функцию только до первой производной, то:

$f(x) = f(x_0) + f'(x_0)(x-x_0) + o((x - x_0)) \approx f(x_0) + f'(x_0)(x-x_0), x \rightarrow x_0$

А если у вас функция от 2 переменных, то тогда:

$f(x, y) \approx f(\theta) + f'_x(\theta)(x-x_0) + f'_y(\theta)(y-y_0), (x, y) \rightarrow \theta, \theta := (x_0, y_0)$.

Теперь, пусть $f(x, y) = \dfrac{x}{y}$. Тогда

$$\begin{align}
\dfrac{x}{y} &\approx \dfrac{x_0}{y_0} + \left(\dfrac{1}{y_0}\right)(x - x_0) + \left(-\dfrac{x_0}{y^2_0}\right)(y-y_0) \\
&= \dfrac{x_0}{y_0} + \dfrac{1}{y_0}\left(x - x_0 - \dfrac{x_0}{y_0}y + x_0\dfrac{y_0}{y_0}\right) \\
&= \dfrac{x_0}{y_0} + \dfrac{1}{y_0}\left(x - \dfrac{x_0}{y_0}y \right)
\end{align}
$$


**Часть 2. Разложение $\dfrac{\overline{X}}{\overline{Y}}$**

Теперь вспомним нашу интересующую ratio-метрику: это $\dfrac{\sum_i^n {X_i}}{\sum_i^n {Y_i}}$, где суммируются все значения по всем пользователям сайта.

Давайте перейдем от суммы к среднему: $\dfrac{\sum_i^n {X_i}}{\sum_i^n {Y_i}} = \dfrac{\overline{X}}{\overline{Y}}$.

Какой интересный факт мы знаем про среднее? По УЗБЧ 
- $\overline{X} \stackrel{p}{\rightarrow} \mathbb{E} X$, 
- $\overline{Y} \stackrel{p}{\rightarrow} \mathbb{E} Y$, 
- $(\overline{X}, \overline{Y}) \stackrel{p}{\rightarrow} (\mathbb{E} X, \mathbb{E} Y)$

Давайте тогда разложим $\dfrac{\overline{X}}{\overline{Y}}$ по формуле Тейлора в точке $(\mathbb{E} X, \mathbb{E} Y)$ относительно 2 переменных $\overline{X}, \overline{Y}$.

$$
    \dfrac{\overline{X}}{\overline{Y}} \approx \dfrac{\mathbb{E} X}{\mathbb{E} Y} + \dfrac{1}{\mathbb{E} Y}\left(\overline{X} - \dfrac{\mathbb{E} X}{\mathbb{E} Y}\overline{Y} \right)
$$

**Часть 3. Разделяем результат на N пользователей**


$$\begin{align}
    \dfrac{\overline{X}}{\overline{Y}} &\approx \dfrac{\mathbb{E} X}{\mathbb{E} Y} + \dfrac{1}{\mathbb{E} Y}\left(\overline{X} - \dfrac{\mathbb{E} X}{\mathbb{E} Y}\overline{Y} \right) \\
    &=\dfrac{\mathbb{E} X}{\mathbb{E} Y} +  \dfrac{1}{\mathbb{E} Y} \cdot \dfrac{1}{N}\sum_{i}^N \left(X_i - \dfrac{\mathbb{E} X}{\mathbb{E} Y} Y_i \right) \\
    &=  \dfrac{1}{N}\sum_{i}^N \left(\dfrac{\mathbb{E} X}{\mathbb{E} Y} + \dfrac{1}{\mathbb{E} Y}\left(X_i - \dfrac{\mathbb{E} X}{\mathbb{E} Y} Y_i \right) \right)
\end{align}
$$

Обозначим за $Z'_i := \dfrac{\mathbb{E} X}{\mathbb{E} Y} + \dfrac{1}{\mathbb{E} Y}\left(X_i - \dfrac{\mathbb{E} X}{\mathbb{E} Y} Y_i \right)$. 

Тогда

$$\begin{align}
    &\dfrac{\overline{X}}{\overline{Y}} \approx \overline Z',\\
    &\mathbb{E} \dfrac{\overline{X}}{\overline{Y}} \approx \mathbb{E} \overline Z'
\end{align}
$$

При этом все $Z'_i$ независимы между собой (так как сами пользователи независимы), и учет зависимости $X, Y$ между собой автоматически учтен в этой выборке. А значит, мы **можем применить t-test к выборке Z'!**

Но мы не знаем $\mathbb{E} X, \mathbb{E}Y$. Поэтому остался последний шаг:

**Часть 4. Заменяем оценку $\mathbb{E} X, \mathbb{E}Y$ на средние**

Тогда финальное $Z_i = \dfrac{\overline X}{\overline Y} + \dfrac{1}{\overline Y}\left(X_i - \dfrac{\overline X}{\overline Y} Y_i \right)$.

Так как $(\overline{X}, \overline{Y}) \stackrel{p}{\rightarrow} (\mathbb{E} X, \mathbb{E} Y)$, то 

$$\begin{align}
Z_i \stackrel{p}{\rightarrow} \dfrac{\mathbb{E} X}{\mathbb{E} Y} + \dfrac{1}{\mathbb{E} Y}\left(X_i - \dfrac{\mathbb{E} X}{\mathbb{E} Y} Y_i \right) = Z'_i
\end{align}
$$

А значит, при больших размерах выборок $\mathbb{E} Z \approx \mathbb{E} Z'$. А значит, $\mathbb{E} Z \approx \mathbb{E} Z' \approx \mathbb{E} \dfrac{\overline{X}}{\overline{Y}}$. Применив T-test к выборке $Z$, мы сможем сравнить $\dfrac{\overline{X}}{\overline{Y}}$.

Давайте реализуем линеаризованную поправку и проверим, работает ли критерий:

In [18]:
def linearisation(numerator, denominator):
    E_num = numpy.mean(numerator)
    E_den = numpy.mean(denominator)
    return E_num / E_den + 1 / E_den * (numerator - E_num / E_den * denominator)

In [20]:
numpy.random.seed(8)

bad_cnt_linearisation = 0
bad_cnt_bootstrap = 0

for _ in tqdm(range(mc_size)):
    test_sample = generate_group_sample_equal_cr(group_size)
    control_sample = generate_group_sample_equal_cr(group_size)
    test_Z = linearisation(test_sample['O_array'], test_sample['message_array'])
    control_Z = linearisation(control_sample['O_array'], control_sample['message_array'])

    # Запускаю критерий и строю доверительный интервал
    left_bound, right_bound = ratio_bootstrap(
        test_sample['O_array'], 
        control_sample['O_array'], 
        test_sample['message_array'], 
        control_sample['message_array']
    )
    pvalue_linearise = ttest_ind(test_Z, control_Z, equal_var=False, alternative='two-sided').pvalue
    
   
    bad_cnt_linearisation += (pvalue_linearise < 0.05)
    bad_cnt_bootstrap += ((left_bound > 0) | (right_bound < 0))


bootstrap_fpr = round(bad_cnt_bootstrap / mc_size, 4)
bootstrap_left, bootstrap_right = proportion_confint(count = bad_cnt_bootstrap, 
                                               nobs = mc_size, alpha=0.05, method='wilson')
print(f"FPR, bootstrap: {round(bootstrap_fpr, 4)},"\
      f" [{round(bootstrap_left, 4)}, {round(bootstrap_right, 4)}]")

linearise_fpr = round(bad_cnt_linearisation / mc_size, 4)
linearise_left, linearise_right = proportion_confint(count = bad_cnt_linearisation, 
                                           nobs = mc_size, alpha=0.05, method='wilson')
print(f"FPR, linearisation: {round(linearise_fpr, 4)},"\
      f" [{round(linearise_left, 4)}, {round(linearise_right, 4)}]")


  0%|          | 0/1000 [00:00<?, ?it/s]

FPR, bootstrap: 0.052, [0.0399, 0.0676]
FPR, linearisation: 0.049, [0.0373, 0.0642]


Да, мы видим, что критерий прекрасно работает! А теперь давайте сравним время работы бутстрапа и линеаризации:

## Сравнение линеаризации и бутстрапа

In [24]:
group_size = 10000
test_sample = generate_group_sample_equal_cr(group_size)
control_sample = generate_group_sample_equal_cr(group_size)

In [27]:
%%time
left_bound, right_bound = ratio_bootstrap(
                                test_sample['O_array'], 
                                control_sample['O_array'], 
                                test_sample['message_array'], 
                                control_sample['message_array']
                            )

CPU times: user 397 ms, sys: 23.9 ms, total: 421 ms
Wall time: 420 ms


In [28]:
%%time
test_Z = linearisation(test_sample['O_array'], test_sample['message_array'])
control_Z = linearisation(control_sample['O_array'], control_sample['message_array'])

pvalue_linearise = ttest_ind(test_Z, control_Z, equal_var=False, alternative='two-sided').pvalue

CPU times: user 0 ns, sys: 1.96 ms, total: 1.96 ms
Wall time: 1.27 ms


Быстрее практически в 400 раз! Что логично, бутстрап работает за `O(NB)` времени (где B &mdash; число бутстрап выборок), а линеаризация работает за `O(N)`.


Также хочется отметить, что линеаризация позволяет написать 1 раз t-test и в зависимости от того, хотите ли вы считать ratio-метрику или считать сразу разницу числителей (например, сразу разницу числа покупок), вы должны подставить в код критерия разные выборки: или выборку числителей или Z-выборку. Что сильно упрощает работу автоматизированной системы проведения AB-тестов, о которой мы говорили в начале раздела.

----
## Альтернативная гипотеза в T-test. Сравниваем среднюю конверсию на одного пользователя.

Часто, когда аналитики сталкиваются с ratio метриками, у них возникает идея вместо гипотезы улучшения конверсии в чат для Авито:

$H_1: \mathbb{E} \dfrac{\sum M^T}{\sum O^T} > \mathbb{E} \dfrac{\sum M^C}{\sum O^C}$

использовать гипотезу улучшения конверсии у каждого отдельного пользователя

$H_1: \mathbb{E} \dfrac{M^T}{O^T} > \mathbb{E} \dfrac{M^C}{O^C}$


Например, в нашей изначальной задаче было бы следующее:



| Id пользователя | Сколько раз зашел на страницу объявления? | Сколько раз начал общение в чате? | конверсия |
|------|----------------------------------------|-----------------------|---------------|
| 1    | 5                                      | 5                     | 1              |
| 2    | 4                                      | 1                     | 1/4            |
| 3    | 0                                      | 0                     | пропускаем     |
| 4    | 6                                      | 4                     | 4/6 |


Мы получили выборку [1, 1/4, 4/6], где значений меньше, чем пользователей: непонятно, что делать с 0 заказов. Итоговое среднее в этой выборке ~47%, что не совпадает с 66% – нашим искомым значением. В этом случае, мы будем считать другую статистику, а не ту, что нас интересовала изначально в задаче!


Посмотрим математически: 
- $O$ &mdash; число открытий объявлений пользователями.
- $M$ &mdash; число заходов в мессенджер пользователями.


Нас интересует посчитать $\mathbb{E} \dfrac{\sum M}{\sum O}$ (общая конверсия из открытия объявления в чат), а в текущем решении мы считаем $\mathbb{E} \dfrac{M}{O}$ (средняя конверсия одного пользователя из объявления в чат, или среднее средних), и это разные вещи!

$$\mathbb{E} \dfrac{\sum M}{\sum O} = \mathbb{E} \dfrac{\overline M}{ \overline O} \neq \mathbb{E} \dfrac{M}{O}$$

Рассмотрим такой пример:

Пусть есть 10 пользователей: 
- у одного 1000 заходов, а количество открытий – 100.
- а у остальных 1 заход и 1 открытие чата.

Конверсия в чаты для всего Авито это 109/1009, что примерно 11%. А средняя конверсия по пользователям – $\dfrac{1/10 + 9}{10} = 91\%$. Это очень большая разница.


**При этом первая конверсия вас интересует больше**: рост $\mathbb{E} \dfrac{\sum M}{\sum O}$ на $X\%$ эквивалентен росту $\mathbb{E} M$ на $X \%$ (вы увеличили число чатов на X процентов), что вас и интересует на самом деле. А для второй дроби вы утверждать такое не можете.

Поэтому текущий метод не подходит: он не считает то, что нам надо.

# Итоги

На текущей лекции мы рассмотрели с вами
- Что такое ratio-метрики. А также показали, что это отличный способ уменьшить шум в данных и получить более мощный критерий.
    - Но надо помнить, что если вы вашим изменением в AB-тесте могли изменить знаменатель, то это стоит дополнительно проверить.
- Решили задачу ratio метрики двумя способами: через бутстрап и через линеаризацию. И показали преимущества линеаризации над бутстрапом. 
- На практике увидели, что такое зависимость в данных и чем она опасна.
- Показали 2 примера, как не надо решать задачу про ratio-метрики.